In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import denumpyize, numpyize, get_template_live_config_vanilla, candidate_to_live_config, calc_spans, get_template_live_config_vanilla, analyze_fills, \
    create_xk, round_dynamic, determine_config_type, round_values
from njit_funcs import calc_emas, calc_long_orders, round_, calc_bankruptcy_price, calc_diff, qty_to_cost
from procedures import dump_live_config, load_live_config, add_argparse_args, prepare_backtest_config, make_get_filepath
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.exchange = 'binance'
        self.symbol = 'XMRUSDT'
        self.market_type = 'futures'
        self.user = 'user_name'
        self.start_date = '2021-04-01'
        self.end_date = '2021-08-23'
        self.starting_balance = 1000.0
        self.starting_configs = ''
        self.base_dir = 'backtests'
config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config['n_days'] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({'timestamp': timestamps, 'qty': qtys, 'price': prices}).set_index('timestamp')
df.price.iloc[::100].plot()

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "config_name": "hand_tuned_vanilla",
    "logging_level": 0,
    "long": {"enabled": True,
          "iprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "iprc_const": 0.995,
          "iqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "iqty_const": 0.005,
          "pbr_limit": 1.8,
          "markup_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "markup_const": 1.005,
          "rprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "rprc_PBr_coeffs": [[-0.0, -0.2]],
          "rprc_const": 0.995,
          "rqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "rqty_const": 1.3,
          "pbr_stop_loss": 0.2},
    "max_span": 60 * 24 * 2,
    "min_span": 60 * 7,
    "n_spans": 3,
    "shrt": {"enabled": True,
          "iprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "iprc_const": 1.01,
          "iqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "iqty_const": 0.005,
          "pbr_limit": 1.8,
          "markup_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "markup_const": 0.995,
          "rprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "rprc_PBr_coeffs": [[0.0, 0.2]],
          "rprc_const": 1.005,
          "rqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
          "rqty_const": 1.3,
          "pbr_stop_loss": 0.2}
}

config['starting_balance'] = 1000.0
config_to_test = {**config, **numpyize(hand_tuned)}
dump_live_config(config_to_test, make_get_filepath(f'tmp/hand_tuned_vanilla.json'))

In [ ]:
sts = time()
fills, info, stats = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
sample_size_ms = dataslice[1][0] - dataslice[0][0]
max_span_ito_n_samples = int(config_to_test['max_span'] * 60 / (sample_size_ms / 1000))
fdf, analysis = analyze_fills(fills, result, dataslice[max_span_ito_n_samples][0], dataslice[-1][0])
pprint.pprint(analysis)
fdf

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
fdf.psize.plot()

In [ ]:
periodic_gain_n_days = 1
groups = fdf.groupby(fdf.timestamp // (1000 * 60 * 60 * 24 * periodic_gain_n_days))
pgs = groups.pnl.sum() / groups.balance.first()
pgs = pgs.reindex(np.arange(pgs.index[0], pgs.index[-1])).fillna(0.0)
pg_mean = pgs.mean()
pgs.plot()
plt.plot([pgs.index[0], pgs.index[-1]], [pg_mean, pg_mean])
print(f'mean {pg_mean:.4f}')
pgs

In [ ]:

# plot MA bands given spans
spans = calc_spans(*[config_to_test[k] for k in ['min_span', 'max_span', 'n_spans']])
print('spans', spans)
emas = pd.DataFrame({str(span): df.price.ewm(span=span * 60, adjust=False).mean() for span in spans})
lband = emas.min(axis=1)
uband = emas.max(axis=1)
df.price.iloc[::100].plot()
uband.iloc[::100].plot()
lband.iloc[::100].plot()


In [ ]:
# see resulting grid
# long only

In [ ]:
xk = create_xk(config_to_test)
spot = xk['spot']
inverse = xk['inverse']
qty_step = xk['qty_step']
price_step = xk['price_step']
min_qty = xk['min_qty']
min_cost = xk['min_cost']
c_mult = xk['c_mult']
pbr_stop_loss = xk['pbr_stop_loss'][0]
pbr_limit = xk['pbr_limit'][0]
iqty_const = xk['iqty_const'][0]
iprc_const = xk['iprc_const'][0]
rqty_const = xk['rqty_const'][0]
rprc_const = xk['rprc_const'][0]
markup_const = xk['markup_const'][0]
iqty_MAr_coeffs = xk['iqty_MAr_coeffs'][0]
iprc_MAr_coeffs = xk['iprc_MAr_coeffs'][0]
rprc_PBr_coeffs = xk['rprc_PBr_coeffs'][0]
rqty_MAr_coeffs = xk['rqty_MAr_coeffs'][0]
rprc_MAr_coeffs = xk['rprc_MAr_coeffs'][0]
markup_MAr_coeffs = xk['markup_MAr_coeffs'][0]

balance = 1000.0
long_psize = 0.0
long_pprice = 0.0
highest_bid = prices[-1]
lowest_ask = prices[-1]
MA_band_lower = prices[-1]
MA_band_upper = prices[-1]
MA_ratios = np.array([1.0, 1.0, 1.0])
available_margin = 2000.0

'''
pbr_limit = config_to_test['long']['pbr_limit']
iqty_const = config_to_test['long']['iqty_const']
iprc_const = config_to_test['long']['iprc_const']
rqty_const = config_to_test['long']['rqty_const']
rprc_const = config_to_test['long']['rprc_const']
rprc_PBr_coeffs = config_to_test['long']['rprc_PBr_coeffs']

'''


orders = []

print("qty          price          psize         pprice        pprice / price ratio    pbr        bkr diff")
for k in range(10):
    long_entry, long_close = calc_long_orders(
        balance,
        long_psize,
        long_pprice,
        highest_bid,
        lowest_ask,
        MA_band_lower,
        MA_band_upper,
        MA_ratios,
        available_margin,

        spot,
        inverse,
        qty_step,
        price_step,
        min_qty,
        min_cost,
        c_mult,
        pbr_stop_loss,
        pbr_limit,
        iqty_const,
        iprc_const,
        rqty_const,
        rprc_const,
        markup_const,
        iqty_MAr_coeffs,
        iprc_MAr_coeffs,
        rprc_PBr_coeffs,
        rqty_MAr_coeffs,
        rprc_MAr_coeffs,
        markup_MAr_coeffs)
    
    
    new_long_psize = long_psize + long_entry[0]
    long_pprice = long_entry[1] * (long_entry[0] / new_long_psize) + long_pprice * (long_psize / new_long_psize)
    long_psize = new_long_psize
    long_pcost = qty_to_cost(long_psize, long_pprice, inverse, c_mult)
    bkr_price = calc_bankruptcy_price(balance,
                          long_psize,
                          long_pprice,
                          0.0,
                          0.0,
                          inverse, c_mult)
    pbr = long_pcost / balance
    if pbr > pbr_limit or long_entry[0] == 0.0:
        break
    bkr_diff = calc_diff(bkr_price, long_entry[1]) if long_entry[1] != 0.0 else 1.0
    print(f"{long_entry[0]: <12} {long_entry[1]: <14} {round_(long_psize, qty_step): <12}  {round_(long_pprice, price_step): <14}"
          f"{round(long_pprice / long_entry[1], 4): <23} {round(pbr, 4): <10} {round(bkr_diff, 6)}")
    orders.append(long_entry)
print()
orders = numpyize(orders)
entry_prices = orders[:,1].astype(float)
print('grid range', entry_prices[0] / entry_prices[-1])
print('dist between grid entries', entry_prices[:-1] / entry_prices[1:])

In [ ]:
n_days_ = 14
nticks = n_days_ * 24 * 60 * 60
print('n ticks', nticks)
edf = pd.DataFrame({p: np.repeat(p, nticks) for p in entry_prices}, index=df.index[-nticks:]).join(df.price.iloc[-nticks:])
edf.iloc[::100].plot()